In [1]:
# Import Libraries
import pandas as pd
import re
import json
from urllib.request import urlopen
import os



In [3]:
#Defining program file path
path = r'C:\Users\12016\Documents\MLCreditModel' 
os.chdir(path)

In [5]:
# Import Data
#US stocks: symbol, name, industry and sector info
#rating- history: Rating agency name, name , date and rating 
df_is_stocks = pd.read_csv('data/uscompanies.csv')
df_ratings = pd.read_csv('data/ratingsinfo.csv')

In [25]:
# Drop NA
#Remove missing values.
df_ratings = df_ratings.dropna()

In [29]:
# Standarize all rating
def standarizeRating(name):
    
    name = re.sub(r'Aaa','AAA', name)
    name = re.sub(r'Aa1', 'AA+', name)
    name = re.sub(r'Aa2', 'AA', name)
    name = re.sub(r'Aa3	', 'AA−', name)
    name = re.sub(r'A1', 'A+', name)    
    name = re.sub(r'A2','A', name)   
    name = re.sub(r'A3', 'A-', name)
    name = re.sub(r'Baa1','BBB+', name)
    name = re.sub(r'Baa2','BBB', name)
    name = re.sub(r'Baa3', 'BBB-', name)
    name = re.sub(r'Ba1','BB+', name)
    name = re.sub(r'Ba2', 'BB', name)
    name = re.sub(r'Ba3', 'BB−', name)
    name = re.sub(r'B1','B+', name)    
    name = re.sub(r'B2','B', name)   
    name = re.sub(r'B3','B−', name)
    name = re.sub(r'Caa1','CCC+', name)
    name = re.sub(r'Caa2','CCC', name)
    name = re.sub(r'Caa3', 'CCC−', name)
    return name

In [27]:
# Apply standarize all rating transformation
df_ratings['Rating'] = df_ratings['Rating'].apply(str).apply(standarizeRating)

In [30]:
# We do not wan't to consider if a Rating was A+ or A. 
# We remove  + and - from ratings
df_ratings['Rating'] = df_ratings['Rating'].str.replace(r'+', '').str.replace(r'-', '')

<ipython-input-30-9d7d27f4933b>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_ratings['Rating'] = df_ratings['Rating'].str.replace(r'+', '').str.replace(r'-', '')


In [31]:
# Filter Ratings
# Exclude non-traditional ratings from the datasets
rating_set = {'A', 'AA', 'AAA', 
              'B', 'BB', 'BBB', 
              'C', 'CC', 'CCC', 
              'D'}

In [32]:
df_ratings = df_ratings[df_ratings['Rating'].isin(rating_set)]

In [33]:
# Filter duplicates in the dataset
df_ratings = df_ratings.drop_duplicates(subset = df_ratings.columns)

df_ratings.groupby('Rating').count()

,Rating Agency Name,Name,Rating Action Date
Rating,,,
A,10810,10810,10810
AA,2501,2501,2501
AAA,269,269,269
B,8518,8518,8518
BB,8258,8258,8258
BBB,15695,15695,15695
C,255,255,255
CC,131,131,131
CCC,3167,3167,3167


In [34]:
# Clean company names
# Remove unwated characters from the names
# We want to merge the datasets df_is_stocks and df_ratings
# The best method is to remove all possible characters that will cause mismatch
# This function will remove all characters and names we don't want


def clean(name):
    
    name = name.lower()
    name = re.sub(r'\.', '', name)
    name = re.sub(r',', '', name)
    name = re.sub(r'^a-z', '', name)
    name = re.sub(r'corporation', '', name)
    name = re.sub(r' corp', '', name)    
    name = re.sub(r' co', '', name)   
    name = re.sub(r'inc', '', name)
    name = re.sub(r'limited', '', name)
    name = re.sub(r'ltd', '', name)
    name = re.sub(r'holdings', '', name)
    name = re.sub(r' holding', '', name)    
    name = re.sub(r'plc', '', name)
    name = re.sub(r'group', '', name)
    name = re.sub(r' ag', '', name)
    name = re.sub(r' sa', '', name)
    name = re.sub(r' pty', '', name)
    name = re.sub(r' international', '', name)
    name = re.sub(r' incorporated', '', name)
    name = re.sub(r' spa', '', name)
    name = re.sub(r' se', '', name)
    name = re.sub(r' lp', '', name)    
    name = re.sub(r' (The)', '', name)    
    name = re.sub(r'The', '', name)    
    name = re.sub(r'LLC', '', name)   
    name = re.sub(r'n.v', '', name)   
    name = name.strip()
    
    return name

In [35]:
# Apply the function in both datasets
df_is_stocks.loc[:, 'Clean_Name'] = df_is_stocks['Name']
df_is_stocks.loc[:, 'Clean_Name'] = df_is_stocks['Name'].apply(clean)
df_ratings.loc[:, 'Clean_Name'] = df_ratings['Name']
df_ratings.loc[:, 'Clean_Name'] = df_ratings['Clean_Name'].apply(str).apply(clean)

In [36]:
# Merge Datasets
df_ratings = pd.merge(df_ratings, df_is_stocks ,on='Clean_Name')

In [37]:
# Clean the table from the merge
df_ratings = df_ratings.rename(columns = {'Name_x': 'Name'}).drop('Name_y', axis = 1)

In [38]:
# Visualise the data
df_ratings.groupby('Rating').count()

,Rating Agency Name,Name,Rating Action Date,Clean_Name,Symbol,Sector,industry
Rating,,,,,,,
A,2482,2482,2482,2482,2482,2193,2193
AA,662,662,662,662,662,602,602
AAA,74,74,74,74,74,64,64
B,1114,1114,1114,1114,1114,994,994
BB,2027,2027,2027,2027,2027,1827,1827
BBB,3825,3825,3825,3825,3825,3329,3329
C,21,21,21,21,21,19,19
CC,49,49,49,49,49,31,31
CCC,370,370,370,370,370,295,295


In [39]:
df_ratings = df_ratings.drop_duplicates(subset = ['Rating Action Date', 'Clean_Name'])

In [40]:
# Now we will get the financial
# We will use the API Financial Modeling Prep
# Create function to get data

def get_jsonparsed_data(url):
    
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)

In [41]:
# List of the symbols/companies 
Symbol_list = list(set(df_ratings.Symbol))

In [42]:
# Remove Unwated characters
Symbol_list = [re.sub(r'\^.+', '', item) for item in Symbol_list]

In [53]:
# Create a dataframe to place the the data
df_company_ratios = pd.DataFrame.from_dict(
        get_jsonparsed_data("https://financialmodelingprep.com/api/v3/ratios/" 
                            + Symbol_list[0] + 
                            "?apikey=ceb428842b06c0eec1a0f12664057f55"))

#df_company_ratios = df_company_ratios[0:0] # empty the dataframe



In [57]:
#df_company_ratios = df_company_ratios[0:0]
#print(df_company_ratios)
df_company_ratios

,symbol,date,period,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,daysOfInventoryOutstanding,operatingCycle,daysOfPayablesOutstanding,...,priceToSalesRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue


In [72]:
# Get every company from the list the financial information
for Symbol in Symbol_list[:200]:
    
    if Symbol.isalpha() == True:
    
        ratios = pd.DataFrame.from_dict(
                get_jsonparsed_data("https://financialmodelingprep.com/api/v3/ratios/" + 
                                    Symbol + 
                                    "?apikey=dd299f272af0e7b72306544f982e6ead"))
    
        frames = [df_company_ratios, ratios]
    
        df_company_ratios = pd.concat(frames)

In [74]:
# Now filter by date
# The api returns every available information from the past
# we want exclusively the data from the year of the rating or the year before
# If the rating was done before june we will take the indicators from the year before
# If it was after june we will take from the following year
df_ratings = df_ratings.rename(columns = {'Rating Action Date':'date'})
df_ratings['date'] = df_ratings['date'].astype('datetime64[ns]')
df_ratings['month_rating'] = df_ratings['date'].dt.month
df_ratings['year_change'] = [1 if x < 7 else 0 for x in df_ratings['month_rating']]
df_ratings['year'] = df_ratings['date'].dt.year
df_ratings['Year'] = df_ratings['year'] - df_ratings['year_change']

df_company_ratios = df_company_ratios.rename(columns = {'symbol':'Symbol'})
df_company_ratios['date'] = df_company_ratios['date'].astype('datetime64[ns]')
df_company_ratios['Year'] = df_company_ratios['date'].dt.year

In [188]:
# merge Datasets
df_ratings = pd.merge(df_ratings, df_company_ratios, on=['Symbol', 'Year']) 

In [189]:
df_ratings_NA = df_ratings

In [190]:
df_ratings_NA = df_ratings_NA = df_ratings_NA.drop_duplicates(['Clean_Name', 'date_y'])

In [193]:
df_ratings_NA.columns

Index(['Rating Agency Name', 'Name', 'date_x', 'Rating', 'Clean_Name',
       'Symbol', 'Sector', 'industry', 'month_rating', 'year_change',
       ...
       'priceToSalesRatio_y', 'priceEarningsRatio_y',
       'priceToFreeCashFlowsRatio_y', 'priceToOperatingCashFlowsRatio_y',
       'priceCashFlowRatio_y', 'priceEarningsToGrowthRatio_y',
       'priceSalesRatio_y', 'dividendYield_y', 'enterpriseValueMultiple_y',
       'priceFairValue_y'],
      dtype='object', length=124)

In [227]:
columns = df_ratings_Clr.columns
letters_index = columns.get_loc("priceSalesRatio_x")
print(letters_index)

64


In [201]:
# clean the dataset - drop columns
df_ratings_Clr =df_ratings_NA.copy()

type(df_ratings_Clr)

pandas.core.frame.DataFrame

In [233]:
# clean the dataset - drop columns
df_ratings_Clr.drop(["Name", "Year", "date_y", "ebtPerEbit_x",
                                    "month_rating", "netIncomePerEBT_x", "year",
                                    "year_change", "industry",
                                    "priceCashFlowRatio_y", "priceEarningsRatio_y",
                                    "priceSalesRatio_x",
                                    "priceToFreeCashFlowsRatio_x",
                                    "priceToOperatingCashFlowsRatio_y",
                                    "priceSalesRatio_x"], axis= 1, inplace =True)



In [234]:
df_ratings_Clr

,Rating Agency Name,date_x,Rating,Clean_Name,Symbol,Sector,period_x,currentRatio_x,quickRatio_x,cashRatio_x,...,dividendPayoutRatio_y,priceBookValueRatio_y,priceToBookRatio_y,priceToSalesRatio_y,priceToFreeCashFlowsRatio_y,priceEarningsToGrowthRatio_y,priceSalesRatio_y,dividendYield_y,enterpriseValueMultiple_y,priceFairValue_y
0,Standard & Poor's Ratings Services,2016-09-29,BB,caci,CACI,Technology,FY,1.631064,1.509086,0.086844,...,None,1.439104,1.439104,0.617752,10.430575,1.376734,0.617752,None,11.289335,1.439104
16,Standard & Poor's Ratings Services,2016-09-14,BBB,kimco realty,KIM,Consumer Services,FY,1.315757,1.315757,0.527203,...,1.251274,1.970959,1.970959,8.848398,17.496544,-0.497617,8.848398,0.045759,20.394091,1.970959
25,Standard & Poor's Ratings Services,2015-11-12,BB,california resources,CRC,Energy,FY,0.821488,0.416529,0.019835,...,-0.003376,-5.982271,-5.982271,2.280383,13.597419,0.010475,2.280383,0.002190,16.445836,-5.982271
32,Standard & Poor's Ratings Services,2016-08-24,D,california resources,CRC,Energy,FY,0.585399,0.396694,0.016529,...,NaN,-1.552172,-1.552172,0.493189,6.650462,0.017920,0.493189,NaN,12.516483,-1.552172
35,Standard & Poor's Ratings Services,2016-10-20,BBB,gatx,GATX,Consumer Services,FY,3.888578,2.995521,1.721725,...,0.262155,1.781495,1.781495,1.692188,-27.682006,0.279463,1.692188,0.028083,9.555350,1.781495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1683,Standard & Poor's Ratings Services,2016-11-28,B,hudbay minerals,HBM,Basic Industries,FY,1.385760,0.976338,0.412138,...,-0.101355,1.043152,1.043152,1.629604,6.516512,0.094483,1.629604,0.001939,6.288077,1.043152
1692,Standard & Poor's Ratings Services,2016-10-25,A,stericycle,SRCL,Basic Industries,FY,1.425097,1.251081,0.081401,...,0.190997,2.315666,2.315666,1.823843,15.804731,-1.029492,1.823843,0.006066,15.730164,2.315666
1701,Standard & Poor's Ratings Services,2016-11-16,BBB,regency centers,REG,Consumer Services,FY,0.932811,0.899536,0.095411,...,1.346643,2.771233,2.771233,11.688539,24.815809,2.313132,11.688539,0.030927,24.331029,2.771233
1710,Moody's Investors Service,2016-10-28,BB,veon,VEON,Public Utilities,FY,0.693915,0.448681,0.448681,...,0.026203,NaN,NaN,0.830703,10.574211,-0.007056,0.830703,0.008265,2.217264,NaN


In [235]:
df_ratings_Clr.columns.to_list()

['Rating Agency Name',
 'date_x',
 'Rating',
 'Clean_Name',
 'Symbol',
 'Sector',
 'period_x',
 'currentRatio_x',
 'quickRatio_x',
 'cashRatio_x',
 'daysOfSalesOutstanding_x',
 'daysOfInventoryOutstanding_x',
 'operatingCycle_x',
 'daysOfPayablesOutstanding_x',
 'cashConversionCycle_x',
 'grossProfitMargin_x',
 'operatingProfitMargin_x',
 'pretaxProfitMargin_x',
 'netProfitMargin_x',
 'effectiveTaxRate_x',
 'returnOnAssets_x',
 'returnOnEquity_x',
 'returnOnCapitalEmployed_x',
 'ebitPerRevenue_x',
 'debtRatio_x',
 'debtEquityRatio_x',
 'longTermDebtToCapitalization_x',
 'totalDebtToCapitalization_x',
 'interestCoverage_x',
 'cashFlowToDebtRatio_x',
 'companyEquityMultiplier_x',
 'receivablesTurnover_x',
 'payablesTurnover_x',
 'inventoryTurnover_x',
 'fixedAssetTurnover_x',
 'assetTurnover_x',
 'operatingCashFlowPerShare_x',
 'freeCashFlowPerShare_x',
 'cashPerShare_x',
 'payoutRatio_x',
 'operatingCashFlowSalesRatio_x',
 'freeCashFlowOperatingCashFlowRatio_x',
 'cashFlowCoverageRatios

In [236]:
# clean the dataset - Rename columns
df_ratings_Clr = df_ratings_Clr.rename({'Clean_Name' :'Name', 'date_x':'Date'})

In [237]:
# Export to CSVDatasets
df_ratings_NA.to_csv (r'data/rating_dataframe_test.csv', index = False, header=True)